In [1]:
# default_exp utils

This contains general utilities used in different modules

# Import

In [2]:
# export
import numpy as np
import torch

In [3]:
from IPython.core.debugger import set_trace

# General

For some reason I can't find a built-in that will reverse and return a list without doing some iterable thing

In [4]:
# export
def reverse(l): return l[::-1]

# General numpy/torch

`torch2np` converts a torch tensor to numpy array since its easy to forget when you need to call `detach` and `cpu` and the order required

In [5]:
# export
def torch2np(A):
    if not isinstance(A, tuple): # Recursion exit condition
        if isinstance(A, torch.Tensor): return A.detach().cpu().numpy()
        else:                           return A
    return tuple(map(torch2np, A))

# Tests

`assert_allclose` checks if two things, `A` and `B`, are close to each other.

In [6]:
# export
def _assert_allclose(A, B, **kwargs):
    if not isinstance(A, tuple): # Recursion exit condition
        try:    assert(np.allclose(A, B, **kwargs))
        except: assert(np.all(A == B))
        return
    
    for a,b in zip(A,B): _assert_allclose(a, b, **kwargs)

In [7]:
# export
def assert_allclose(A, B, **kwargs):
    A, B = map(torch2np, [A, B]) # Conversion needed if torch tensor is on gpu, otherwise np.allclose fails
    _assert_allclose(A, B, **kwargs)

In [8]:
A = torch.rand((4,3)).cuda(); 
B = np.random.normal(size=(4,3))
A, B

(tensor([[0.9530, 0.5480, 0.3687],
         [0.9981, 0.8400, 0.0459],
         [0.4373, 0.8938, 0.3349],
         [0.5041, 0.9184, 0.0670]], device='cuda:0'),
 array([[ 0.35471579,  0.90508074,  0.5217619 ],
        [ 1.4196832 ,  0.11504157,  1.32236272],
        [-1.2066323 ,  1.02589301,  2.28333353],
        [-2.16652503,  0.37212147, -0.20945945]]))

In [9]:
assert_allclose(A, A+1e-5, atol=1e-5)
assert_allclose((A, (B, 1., 'test')), (A+1e-5, (B+1e-5, 1.+1e-5, 'test')), atol=1e-5)

Since we have multiple functions that should work for torch and numpy, we should have a way to test for both without having to write duplicate tests

In [10]:
# export
def assert_allclose_f(f, x, y, **kwargs):
    if not isinstance(x, tuple): x = (x,)
    assert_allclose(f(*x), y, **kwargs)

In [11]:
# export 
def assert_allclose_f_ttn(f, x, y, **kwargs): # ttn == "torch, then numpy"
    assert_allclose_f(f, x, y, **kwargs) # Torch test
    x, y = map(torch2np, [x,y])
    assert_allclose_f(f, x, y, **kwargs) # Numpy test

# General point stuff

`psify` decorator will allow functions designed for multiple points to work for single point inputs

In [12]:
# export
def psify(f):
    def _psify(ps, *args, **kwargs):
        single = len(ps.shape) == 1
        if single: ps = ps[None]
        ps = f(ps, *args, **kwargs)
        if single: ps = ps[0]
        return ps
    return _psify

`augment` will add ones to points; useful for affine and homography xforms

In [13]:
# export
@psify
def augment(ps):
    if isinstance(ps, np.ndarray): 
        return np.c_[ps, np.ones(len(ps), dtype=ps.dtype)]
    else:
        return torch.cat([ps, ps.new_ones((len(ps), 1))], dim=1)

In [14]:
ps = torch.tensor([[0.1940, 0.2536],
                   [0.2172, 0.1626],
                   [0.9834, 0.2700],
                   [0.5324, 0.7137]]).cuda()
assert_allclose_f_ttn(augment, ps, torch.tensor([[0.1940, 0.2536, 1.0000],
                                                 [0.2172, 0.1626, 1.0000],
                                                 [0.9834, 0.2700, 1.0000],
                                                 [0.5324, 0.7137, 1.0000]]).cuda())

`deaugment` will remove last column; might wanna add check to make sure column contains ones

In [15]:
# export
@psify
def deaugment(ps): return ps[:, 0:-1]

In [16]:
ps = torch.tensor([[0.1940, 0.2536, 1.0000],
                   [0.2172, 0.1626, 1.0000],
                   [0.9834, 0.2700, 1.0000],
                   [0.5324, 0.7137, 1.0000]]).cuda()
assert_allclose_f_ttn(deaugment, ps, torch.tensor([[0.1940, 0.2536],
                                                   [0.2172, 0.1626],
                                                   [0.9834, 0.2700],
                                                   [0.5324, 0.7137]]).cuda())

`normalize` will divide by last column and remove it

In [17]:
# export
@psify
def normalize(ps): return deaugment(ps/ps[:, [-1]])

`unitize` will make norm of each point 1

In [18]:
# export
@psify
def unitize(ps): return ps/np.linalg.norm(ps, axis=1, keepdims=True)

`pmm` is point matrix multiplication

In [19]:
# export
@psify
def pmm(ps, A, aug=False):
    if aug: ps = augment(ps)
    ps = (A@ps.T).T
    if aug: ps = normalize(ps) # works for both affine and homography transforms
    return ps

In [20]:
ps = torch.tensor([[0.1940, 0.2536],
                   [0.2172, 0.1626],
                   [0.9834, 0.2700],
                   [0.5324, 0.7137]]).cuda()
A = torch.tensor([[0.9571, 0.5551],
                  [0.8914, 0.2626]]).cuda()
assert_allclose_f_ttn(pmm, (ps,A), torch.tensor([[0.3265, 0.2395],
                                                 [0.2981, 0.2363],
                                                 [1.0911, 0.9475],
                                                 [0.9057, 0.6620]]).cuda(), atol=1e-4)

# Bounding box stuff

`ps_bb` is points bounding box

In [21]:
# export
def ps_bb(ps): return np.stack([np.min(ps, axis=0), np.max(ps, axis=0)])

`array_bb` is array bounding box

In [22]:
# export
def array_bb(arr): return np.array([[0,0], [arr.shape[1]-1, arr.shape[0]-1]])

`bb_sz` returns the size of a bounding box

In [23]:
# export
def bb_sz(bb):
    assert_allclose(bb.dtype, np.int)
    return np.array([bb[1,1]-bb[0,1]+1, bb[1,0]-bb[0,0]+1])

`bb_grid` is bounding box grid; i,j is swapped to x,y

In [24]:
# export
def bb_grid(bb):
    assert_allclose(bb.dtype, np.int)
    return reverse(np.mgrid[bb[0,1]:bb[1,1]+1, bb[0,0]:bb[1,0]+1])

`bb_array` applies bounding box to array and returns the sub array

In [25]:
# export 
def bb_array(arr, bb): 
    assert_allclose(bb.dtype, np.int)
    return arr[bb[0,1]:bb[1,1]+1, bb[0,0]:bb[1,0]+1]

# Other point stuff

`grid2ps` converts grid to points

In [26]:
# export
def grid2ps(X, Y, order='C'): return np.c_[X.ravel(order), Y.ravel(order)]

`array_ps` is array points

In [27]:
# export
def array_ps(arr): return grid2ps(*bb_grid(array_bb(arr)))

# Transforms

`condition_mat` is typically used to "condition" points to improve conditioning; its inverse is usually applied afterwards. It sets the mean of the points to zero and the average distance to `sqrt(2)`. I use the term "condition" here so I don't get confused with "normalization" which is used above.

In [28]:
# export
def condition_mat(ps):
    xs, ys = ps[:, 0], ps[:, 1]
    mean_x, mean_y = xs.mean(), ys.mean()
    s_m = np.sqrt(2)*len(ps)/(np.sqrt((xs-mean_x)**2+(ys-mean_y)**2)).sum()
    return np.array([[s_m,   0, -mean_x*s_m],
                     [  0, s_m, -mean_y*s_m],
                     [  0,   0,           1]])

In [29]:
# export
def condition(ps):
    T = condition_mat(ps)
    return pmm(ps, T, aug=True), T

In [30]:
ps = array_ps(np.zeros((3,2)))
assert_allclose(ps, np.array([[0, 0],
                              [1, 0],
                              [0, 1],
                              [1, 1],
                              [0, 2],
                              [1, 2]]))
assert_allclose(condition_mat(ps), np.array([[ 1.55063424,  0.        , -0.77531712],
                                             [ 0.        ,  1.55063424, -1.55063424],
                                             [ 0.        ,  0.        ,  1.        ]]))

`homography` estimates a homography between two sets of points

In [31]:
# export
def homography(ps1, ps2):    
    # Condition and augment points
    (ps1_cond, T1), (ps2_cond, T2) = map(condition, [ps1, ps2])
    ps1_cond, ps2_cond = map(augment, [ps1_cond, ps2_cond])
    
    # Form homogeneous system
    L = np.r_[np.c_[ps1_cond, np.zeros_like(ps1_cond), -ps2_cond[:, 0:1]*ps1_cond],
              np.c_[np.zeros_like(ps1_cond), ps1_cond, -ps2_cond[:, 1:2]*ps1_cond]]
    
    # Solution is the last row of V
    _,_,V = np.linalg.svd(L)
    H12_cond = V[-1, :].reshape(3,3)
    
    # Undo conditioning
    H12 = np.linalg.inv(T2)@H12_cond@T1
    H12 /= H12[2,2] # Sets H12[2,2] to 1
    return H12

`approx_R` gives the nearest rotational approximation to the input matrix (I believe frobenium norm). Note that for a proper rotation determinant must be +1, which is checked after.

In [32]:
# export
def approx_R(R):
    [U,_,V] = np.linalg.svd(R)
    R = U@V
    if not np.isclose(np.linalg.det(R), 1):
        R = np.full((3,3), np.nan)
    return R

In [33]:
# export
def Rt2M(R, t):
    M = torch.cat([R, t[:,None]], dim=1)
    M = torch.cat([M, M.new_tensor([[0,0,0,1]])])
    return M

In [34]:
# export
def M2Rt(M): return M[0:3,0:3], M[0:3,3]

In [35]:
# export
def invert_rigid(M):
    R, t = M2Rt(M)
    return Rt2M(R.T, -R.T@t)

In [36]:
# export
def mult_rigid(M1, M2):
    R1, t1 = M2Rt(M1)
    R2, t2 = M2Rt(M2)
    return Rt2M(R1@R2, R1@t2+t1)

# Ellipse stuff

`sample_2pi` prevents accidentally resampling 2pi twice by linspacing with an additional sample and then removing the last sample

In [37]:
# export
def sample_2pi(num_samples): return np.linspace(0, 2*np.pi, num_samples+1)[:-1]

In [38]:
# export
def sample_ellipse(h, k, a, b, alpha, num_samples):
    sin, cos = np.sin, np.cos    
    
    thetas = sample_2pi(num_samples)
    return np.c_[a*cos(alpha)*cos(thetas) - b*sin(alpha)*sin(thetas) + h,
                 a*sin(alpha)*cos(thetas) + b*cos(alpha)*sin(thetas) + k]

In [39]:
# export
def ellipse2conic(h, k, a, b, alpha):
    sin, cos = np.sin, np.cos
    
    A = a**2*sin(alpha)**2 + b**2*cos(alpha)**2
    B = 2*(b**2 - a**2)*sin(alpha)*cos(alpha)
    C = a**2*cos(alpha)**2 + b**2*sin(alpha)**2
    D = -2*A*h - B*k
    E = -B*h - 2*C*k
    F = A*h**2 + B*h*k + C*k**2 - a**2*b**2

    return np.array([[  A, B/2, D/2],
                     [B/2,   C, E/2],
                     [D/2, E/2,   F]])

In [40]:
# export
def conic2ellipse(Aq):
    sqrt, abs, arctan, pi = np.sqrt, np.abs, np.arctan, np.pi
    
    A = Aq[0, 0]
    B = 2*Aq[0, 1]
    C = Aq[1, 1]
    D = 2*Aq[0, 2]
    E = 2*Aq[1, 2]
    F = Aq[2, 2]

    # Return nans if input conic is not ellipse
    if np.any(~np.isfinite(Aq.ravel())) or np.isclose(B**2-4*A*C, 0) or B**2-4*A*C > 0:
        return np.full(5, np.nan)

    # Equations below are from https://math.stackexchange.com/a/820896/39581

    # "coefficient of normalizing factor"
    q = 64*(F*(4*A*C-B**2)-A*E**2+B*D*E-C*D**2)/(4*A*C-B**2)**2

    # distance between center and focal point
    s = 1/4*sqrt(abs(q)*sqrt(B**2+(A-C)**2))

    # ellipse parameters
    h = (B*E-2*C*D)/(4*A*C-B**2)
    k = (B*D-2*A*E)/(4*A*C-B**2)
    a = 1/8*sqrt(2*abs(q)*sqrt(B**2+(A-C)**2)-2*q*(A+C))
    b = sqrt(a**2-s**2)
    # Get alpha; note that range of alpha is [0, pi)
    if np.isclose(q*A-q*C, 0) and np.isclose(q*B, 0):     alpha = 0 # Circle
    elif np.isclose(q*A-q*C, 0) and q*B > 0:              alpha = 1/4*pi
    elif np.isclose(q*A-q*C, 0) and q*B < 0:              alpha = 3/4*pi
    elif q*A-q*C > 0 and (np.isclose(q*B, 0) or q*B > 0): alpha = 1/2*arctan(B/(A-C))
    elif q*A-q*C > 0 and q*B < 0:                         alpha = 1/2*arctan(B/(A-C)) + pi
    elif q*A-q*C < 0:                                     alpha = 1/2*arctan(B/(A-C)) + 1/2*pi
    else: raise RuntimeError('"Impossible" condition reached; please debug')

    return h, k, a, b, alpha

# General image processing

I think `conv2d` is actually cross correlation...

In [41]:
# export
def conv2d(arr, kernel, **kwargs):
    assert_allclose(arr.dtype, kernel.dtype)
    _conv2d = torch.nn.functional.conv2d
    arr, kernel = map(torch.tensor, [arr, kernel])
    return torch2np(_conv2d(arr[None,None], kernel[None, None], **kwargs)).squeeze(axis=(0,1))

In [42]:
# export
def grad_array(arr):
    kernel_sobel = np.array([[-0.1250, 0, 0.1250],
                             [-0.2500, 0, 0.2500],
                             [-0.1250, 0, 0.1250]], dtype=arr.dtype)
    arr = np.pad(arr, 1, mode='edge')
    return [conv2d(arr, kernel) for kernel in [kernel_sobel, kernel_sobel.T]]

# Optimization stuff

`wlstsq` is weighted least squares

In [43]:
# export
def wlstsq(A, b, W=None):
    # Weights should be a diagonal matrix with sqrt of the input weights
    if W is not None:
        W = np.sqrt(W.ravel())
        A, b = A*W[:,None], b*W
    return np.linalg.lstsq(A, b, rcond=None)

# Build

In [44]:
%%javascript
IPython.notebook.save_notebook()

<IPython.core.display.Javascript object>

In [45]:
!nbdev_build_lib

Converted README.ipynb.
Converted calib.ipynb.
Converted cb_geom.ipynb.
Converted control_refine.ipynb.
Converted fiducial_detect.ipynb.
Converted image.ipynb.
Converted modules.ipynb.
Converted plot.ipynb.
Converted utils.ipynb.
